In [1]:
from tensorflow import keras
from tensorflow.keras.layers import *
from prepare_data import *
from datetime import datetime

In [2]:
dataset = Dataset.carabid
dataset_name = str(dataset).split(".")[1]
current_time = datetime.now().strftime("%d%m%Y-%H%M%S")

In [3]:
# prep an image generator which makes inputs in sets of three
raw_train, raw_val = prep_dataset(dataset, 8)
train_gen, val_gen = prep_multi_img_ensemble_dataset(dataset, raw_train, raw_val, 8, 3)

Found 63364 files belonging to 291 classes.
Using 53860 files for training.
Found 63364 files belonging to 291 classes.
Using 9504 files for validation.


In [4]:
# load 3 inception extractors: one for each input image
extractor_path = f"../model-saves/extractors/{dataset_name}/{dataset_name.upper()}-EXTRACTOR/extractor/savefile.hdf5"
extractor1 = keras.models.load_model(extractor_path).layers[0].layers[-1]
extractor1.trainable = True
for layer in extractor1.layers:
    layer._name += "_1"

extractor2 = keras.models.load_model(extractor_path).layers[0].layers[-1]
extractor2.trainable = False
for layer in extractor2.layers:
    layer._name += "_2"

extractor3 = keras.models.load_model(extractor_path).layers[0].layers[-1]
extractor3.trainable = False
for layer in extractor3.layers:
    layer._name += "_3"

In [5]:
# concatenate extracted features and classify them
concat_layer = keras.layers.concatenate([extractor1.output, extractor2.output, extractor3.output])
dropout_layer = Dropout(0.5)(concat_layer)
dense_layer = Dense(1000, activation='relu')(dropout_layer)
predictions = keras.layers.Dense(train_gen.num_classes(), activation='softmax')(dense_layer)

In [6]:
classifier_model = keras.Model(inputs=[extractor1.input, extractor2.input, extractor3.input], outputs=predictions)
classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
# callbacks to save the logs and model each epoch
logdir = "../logs/unfiltered/3_img_ensemble_{0}_{1}/classifier".format(str(dataset), current_time)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model_path = "../model-saves/unfiltered/3_img_ensemble_{0}_{1}/classifier/savefile.hdf5".format(str(dataset), current_time)
model_save_callback = keras.callbacks.ModelCheckpoint(filepath=model_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [8]:
# callback to shuffle the dataset and image sets each epoch
class ShuffleCallback(keras.callbacks.Callback):
    def __init__(self, generator):
        self._generator = generator
        
    def on_epoch_end(self, epoch, logs=None):
        self._generator.shuffle()
    
train_shuffle_callback = ShuffleCallback(train_gen)
val_shuffle_callback = ShuffleCallback(val_gen)

In [9]:
classifier_model.fit(train_gen, validation_data=val_gen, callbacks=[tensorboard_callback, model_save_callback, train_shuffle_callback, val_shuffle_callback], epochs=20)

Epoch 1/20
2233/2233 [==============================] - ETA: 0s - loss: 4.4400 - accuracy: 0.2658
Epoch 1: val_accuracy improved from -inf to 0.73064, saving model to ../model-saves/unfiltered/3_img_ensemble_Dataset.carabid_18052022-131857/classifier\savefile.hdf5
2233/2233 [==============================] - 379s 161ms/step - loss: 4.4400 - accuracy: 0.2658 - val_loss: 1.1309 - val_accuracy: 0.7306
Epoch 2/20
2233/2233 [==============================] - ETA: 0s - loss: 1.3800 - accuracy: 0.6550
Epoch 2: val_accuracy improved from 0.73064 to 0.89102, saving model to ../model-saves/unfiltered/3_img_ensemble_Dataset.carabid_18052022-131857/classifier\savefile.hdf5
2233/2233 [==============================] - 353s 158ms/step - loss: 1.3800 - accuracy: 0.6550 - val_loss: 0.3935 - val_accuracy: 0.8910
Epoch 3/20
2233/2233 [==============================] - ETA: 0s - loss: 0.6841 - accuracy: 0.8044
Epoch 3: val_accuracy improved from 0.89102 to 0.93917, saving model to ../model-saves/unfilter

KeyboardInterrupt: 